In [20]:
# Dependencies
import os
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
from sqlalchemy import Column, Integer, String, Float 
import psycopg2

In [40]:
rds_connection_string = "<insert user name>:<insert password>@localhost:5432/customer_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [43]:
athletes_events_file = os.path.join(r"C:\Users\Lenovo_7\Desktop\Class_Folders\ETL\athlete_events.csv")
regions_file = os.path.join(r"C:\Users\Lenovo_7\Desktop\Class_Folders\ETL\noc_regions.csv")
variables_file = os.path.join(r"C:\Users\Lenovo_7\Desktop\Class_Folders\ETL\country_profile_variables.csv")
kiva_file = os.path.join(r"C:\Users\Lenovo_7\Desktop\Class_Folders\ETL\kiva_country_profile_variables.csv")
ses_file = os.path.join(r"C:\Users\Lenovo_7\Desktop\Class_Folders\ETL\country_SES_1880-2010_1.csv")



In [75]:
athletes_df = pd.read_csv(athletes_events_file)
athletes_df.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN


In [23]:
# Get a list of all of our columns for easy reference
athletes_df.columns

Index(['ID', 'Name', 'Sex', 'Age', 'Height', 'Weight', 'Team', 'NOC', 'Games',
       'Year', 'Season', 'City', 'Sport', 'Event', 'Medal'],
      dtype='object')

In [74]:
regions_df = pd.read_csv(regions_file)
noc_df = pd.DataFrame(regions_df, columns = ["NOC", "region"])
noc_df.head()

,NOC,region
0,AFG,Afghanistan
1,AHO,Curacao
2,ALB,Albania
3,ALG,Algeria
4,AND,Andorra


In [69]:
olympic = pd.DataFrame(athletes_df, columns = ["Team", "NOC", "Year", "Season", "City", "Sport", "Event", "Medal"])
olympic.head()


,Team,NOC,Year,Season,City,Sport,Event,Medal
0,China,CHN,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,China,CHN,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,Denmark,DEN,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,Denmark/Sweden,DEN,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,Netherlands,NED,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN


In [72]:
olympic = olympic.rename(columns={"NOC": "Country_ID"})
olympic = olympic.dropna()
olympic.to_csv(r"C:\Users\Lenovo_7\Desktop\Class_Folders\ETL\olympic_cleaned.csv")

In [64]:
ses_df = pd.read_csv(ses_file, encoding=  "ISO-8859-1")
ses = pd.DataFrame(ses_df, columns = ["wbid", "country", "year", "SES", "gdppc"])
ses = ses.rename(columns={"wbid": "Country_ID"})
ses

,Country_ID,country,year,SES,gdppc
0,AFG,Afghanistan,1970,3.474212,709.00000
1,AFG,Afghanistan,1920,26.968016,731.75677
2,AFG,Afghanistan,1990,1.269530,604.00000
3,AFG,Afghanistan,1960,15.763076,739.00000
4,AFG,Afghanistan,2000,2.061114,565.00000
5,AFG,Afghanistan,2010,5.676400,1662.80350
6,AFG,Afghanistan,1880,37.957447,585.46509
7,AFG,Afghanistan,1940,14.320977,673.91895
8,AFG,Afghanistan,1890,29.591391,635.93024
9,AFG,Afghanistan,1980,3.465048,690.00000


In [ ]:
--Just storing table schema here
CREATE TABLE athlete_events (
    id INT NOT NULL,
    Name VARCHAR(50) NOT NULL,
	Sex VARCHAR(10) NOT NULL,
	Age INT NOT NULL,
	Height VARCHAR(50) NOT NULL,
	Weight INT NOT NULL,
	Team VARCHAR(50) NOT NULL,
	NOC VARCHAR(5) NOT NULL,
	Games VARCHAR(50) NOT NULL,
	Year INT NOT NULL,
	Season VARCHAR(50) NOT NULL,
	City VARCHAR(50) NOT NULL,
	Sport VARCHAR(50) NOT NULL,
	Event VARCHAR(50) NOT NULL,
	Medal VARCHAR(10) NOT NULL
);

CREATE TABLE noc_regions (
    NOC VARCHAR(5) NOT NULL,
    region VARCHAR(50) NOT NULL,
    notes VARCHAR(50)
);